# Simple Demo about MongoDB FDW of PostgreSQL
## MongoDB connection in Python

In [1]:
import os
from pymongo import MongoClient
from datetime import datetime
from random import randint
client =  MongoClient("mongodb://mongo:27017")
db = client.primer
ds = db.dataset
result = ds.delete_many({})
ds.insert_many([{'i': i} for i in range(10000)]).inserted_ids
ds.count()

10000

## PostgreSQL connection in Python

In [9]:
import psycopg2
conn = psycopg2.connect(database="gis", user="mongo", password="mongotest", host="db", port="5432")
cursor = conn.cursor()
cursor.execute("select relname from pg_class where relkind='r' and relname !~ '^(pg_|sql_)';")
print(cursor.fetchall() )
conn.close() 

[('addrfeat',), ('spatial_ref_sys',), ('edges',), ('faces',), ('topology',), ('place_lookup',), ('layer',), ('geocode_settings',), ('geocode_settings_default',), ('direction_lookup',), ('secondary_unit_lookup',), ('state_lookup',), ('street_type_lookup',), ('county_lookup',), ('countysub_lookup',), ('zip_lookup_all',), ('zip_lookup_base',), ('zip_lookup',), ('county',), ('state',), ('place',), ('zip_state',), ('zip_state_loc',), ('cousub',), ('featnames',), ('addr',), ('zcta5',), ('loader_platform',), ('loader_variables',), ('loader_lookuptables',), ('tract',), ('tabblock',), ('bg',), ('pagc_gaz',), ('pagc_lex',), ('pagc_rules',)]
